In [2]:
import numpy as np
import pandas as pd
import math
import networkx as nx
import csv
#from collections import defaultdict
from statistics import median, mean
#import re
import queue as Q
import threading
from IPython.display import clear_output

In [3]:
def open_dict(vocabularyFile):
        cats = open(vocabularyFile, 'r', encoding = "utf8").readlines()
        cats2 = {}
        for cat in cats:    
            templ = []
            for x in cat.split():
                templ.append(x.strip("[").strip("]").strip("'").strip(",").rstrip("''"))
            cats2[templ[0]] = templ[1:]
        return cats2

#function to save our vocabulary file to disk        
def save_dict(vocabulary,fileName="output.csv"):
    with open(fileName,'wb') as vfile:
        for i in vocabulary.keys():
            vfile.write(str(i).encode())
            vfile.write(str('\t').encode())
            vfile.write(str(vocabulary[i]).encode())
            vfile.write('\n'.encode())
        vfile.close()

In [5]:
#opening the file that countains the information of our graph
nodesfile = np.loadtxt('wiki-topcats-reduced.txt', delimiter="\t", dtype =int)

In [4]:
#counting the numbers of self loops
self_count = 0
for row in nodesfile:
    if row[0] == row[1]:
        self_count += 1
self_count

777

In [5]:
# checking if graph is directed by seeing if the first node occurs the same number of times in both columns. 
# if the number of occurences is the same, we have a undirected graph. If not, its directed.  
dir_count1 = 0
dir_count2 = 0
for node1 in nodesfile:
    if nodesfile[0][0] == node1[0]:
        dir_count1 +=1
    if nodesfile[0][0] == node1[1]:
        dir_count2 +=1
if dir_count1 != dir_count2:
    print("directed graph")
else:
    print("no directed graph")


directed graph


In [6]:
# Creating the grpah using the networkx library (as the TA's said on slack that we can use 
# networkx for this purpose). We choose set the graph type to MultiDiGraph as we already prooved
# that there are self loops and that we have a directed graph.
# By creating the edges from our file, the nodes will automatically be created by networkx.
graph = nx.MultiDiGraph()
for nodes in nodesfile:
    graph.add_edge(nodes[0], nodes[1])

In [5]:
nx.info(graph)

'Name: \nType: MultiDiGraph\nNumber of nodes: 461193\nNumber of edges: 2645247\nAverage in degree:   5.7357\nAverage out degree:   5.7357'

In [21]:
#getting the number of nodes by getting a list of unique values and the get the length of this list
no_nodes = len(np.unique(nodesfile))
no_nodes

461193

In [22]:
# the number of all edges we can also retrieve by seeing how many rows our array has, as every row represents one edge
no_edges = len(nodesfile)
no_edges

2645247

We first tried to evaluate the number of edges and nodes directly from the given file. As the outputs above show, there are no differences in the number of edges we assumed and the number of edges counted by networkx. 

In [144]:
#calculating the average degree by dividing the total number of edges (number of rows in our array) by the number of nodes (unique values in our array)
avg_degree = no_edges/no_nodes
avg_degree

5.735661642739591

As you can see here, the result of our calculation for the average doesn't differ lot from the output nx.info() gave us.

In [145]:
#calculating the density
dens = (no_edges)/(no_nodes*(no_nodes-1))
dens

1.2436602635647606e-05

Calculating the density using the number of edges we read from the file earlier. As a value of 1 indicates a very dense graph and our result for the density is very low value, we conclude that it is a very sparse graph. 

In [35]:
categories = open('wiki-topcats-categories.txt', 'r', encoding = "utf8").readlines()

Creating a list of unique nodes to be used later. 

In [6]:
nodes = np.unique(nodesfile)

Creating a dictionary out of the given list by removing "Category:" and "\n" and the splitting the lines. The first value of each line is the name of the category (we used it as key for our dictionary). The following values are the nodes contained in this category.

In [37]:
cats = {}
for line in categories:
    temp = line.strip("Category:").strip("\n").split()
    temp[0] = temp[0].strip(";")
    cats[temp[0]] = temp[1:]

In [38]:
len(cats)

17364

Before we clean up our dictionary we already remove all categories that contain less than 3500 entries. This will spare us some time later on as there will be less values/categories we have to check.

In [39]:
cats2 = {}
for key in cats.keys():
    if len(cats[key]) > 3500:
        cats2[key] = cats[key]

In [40]:
len(cats2)

35

In the following step we created a second dictionary by checking if the values for every key are contained in our list of unique nodes we created earlier

In [41]:
for key in cats2.keys():
    temp = []
    for node in cats2[key]:
        if int(node) in nodes:
            temp.append(node)
    cats2[key] = temp


As you can see after cleaning up, six of our categories contain less then 3500 entry's again.

In [42]:
i = 0
for keys in cats2.keys():
    if len(cats2[keys]) < 3500:
        i+=1
print(i)

6


In [28]:
save_dict(cats2,fileName="wiki-topcats-categories_modified.txt")

In [7]:
cats2 = open_dict("wiki-topcats-categories_modified.txt")

In [1]:
pagenames = open('wiki-topcats-page-names.txt', 'r', encoding = "utf8").readlines()

In [30]:
pagenames[0]

'0 Chiasmal syndrome\n'

In [31]:
pages = {}
for page in pagenames:
    page.strip("\n").split()[0]
    if int(page.strip("\n").split()[0]) in nodes:
        pages[page.strip("\n").split()[0]] = " ".join(page.strip("\n").split()[1:])

In [32]:
save_dict(pages,fileName='wiki-topcats-page-names_modified.txt')

In [33]:
pages['52']

'Hung Huang'

For the usage in the further steps we append the categories to the nodes. If the node belongs to a category, the value will be set to "True". Otherwise is will be "False". 

In [23]:
for key in cats2.keys():
    for node in graph.nodes:
        graph.node[node][key] = False
        graph.node[node]['score'] = 0
    temp = cats2[key]
    for node in temp:
        graph.node[int(node)][key] = True

In [9]:
graph.node[22860]

{'English_footballers': True,
 'The_Football_League_players': False,
 'Association_football_forwards': False,
 'Association_football_goalkeepers': False,
 'Association_football_midfielders': False,
 'Association_football_defenders': True,
 'Living_people': False,
 'Year_of_birth_unknown': False,
 'Harvard_University_alumni': False,
 'Major_League_Baseball_pitchers': False,
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': False,
 'Indian_films': False,
 'Year_of_death_missing': False,
 'English_cricketers': False,
 'Year_of_birth_missing_(living_people)': False,
 'Rivers_of_Romania': False,
 'Main_Belt_asteroids': False,
 'Asteroids_named_for_people': False,
 'English-language_albums': False,
 'English_television_actors': False,
 'British_films': False,
 'English-language_films': False,
 'American_films': False,
 'Fellows_of_the_Royal_Society': False,
 'People_from_New_York_City': False,
 'American_Jews': False,
 'American_television_actors': False,
 'American_fil

In [ ]:


def bfs(graph, inp_cat, inp_node, dest_cat, out_q):
    queue = Q.Queue()
    queue.put([inp_node, 0])
    visited = {}
    sh_path = np.inf
    for x in graph.nodes:
        visited[x] = False
    while queue.empty() != True:
        current = queue.get()
        if graph.node[current[0]][dest_cat] == True and graph.node[current[0]][inp_cat] != True:
            visited[current[0]]= True
            sh_path = current[1]
            #print('shortest path from ', inp_node, ' to ', current[0], ' found (dist = ', current[1], ')')
            queue.queue.clear()
        else:            
            for i in graph.successors(current[0]):
                if visited[i] != True:
                    queue.put([i, current[1]+1])
            visited[current[0]]= True        
    out_q.put([inp_node, sh_path])
    
def run_bfs(start_cat, graph, categories):
    inp_nodes = [cat_nodes for cat_nodes in graph.nodes if graph.node[cat_nodes][start_cat]== True]
    medians = {}
    for cat in categories:
        sh_paths = {}
        if cat != start_cat:
            dest_cat = cat
            start_q = Q.Queue()
            out_q = Q.Queue()
            for start_node in inp_nodes:
                start_q.put(start_node)
            while not start_q.empty(): 
                if threading.active_count() <= 50:
                    current_t = start_q.get()
                    t = threading.Thread(target=bfs, args=(graph, start_cat, current_t, dest_cat, out_q), daemon= True).start()
                    start_q.task_done()
            while not out_q.empty():
                out_p = out_q.get()
                sh_paths[out_p[0]] = out_p[1]
                out_q.task_done()
            start_q.join()
            sum_vals = 0
            i = 0
            for x in sh_paths.values():
                if x != np.inf:
                    i+=1
                    sum_vals += x        
            medians[cat] = [median(sh_paths.values()), sum_vals/i]
    return medians
    
smallest = run_bfs('Year_of_birth_unknown', graph, cats2.keys())

In [ ]:
save_dict(smallest, 'results.csv')

In [40]:
smalles = open_dict('results.csv')

In [41]:
smalles 

{'English_footballers': ['4.0', '3.8982960596379126'],
 'The_Football_League_players': ['4', '3.986140724946695'],
 'Association_football_forwards': ['4', '3.668082846521508'],
 'Association_football_goalkeepers': ['4.0', '4.000533049040512'],
 'Association_football_midfielders': ['4', '3.5979819437068508'],
 'Association_football_defenders': ['5.0', '4.290722761596548'],
 'Living_people': ['2.0', '2.027277406073083'],
 'Year_of_birth_unknown': ['2.0', '2.027277406073083'],
 'Harvard_University_alumni': ['3.0', '2.779947229551451'],
 'Major_League_Baseball_pitchers': ['5.0', '4.369330453563715'],
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': ['3.0',
  '3.179175475687104'],
 'Indian_films': ['5', '4.318546231961518'],
 'Year_of_death_missing': ['4', '3.3719353155972875'],
 'English_cricketers': ['4.0', '3.7357594936708862'],
 'Year_of_birth_missing_(living_people)': ['3.0', '2.9715489989462593'],
 'Rivers_of_Romania': ['4.0', '3.797542735042735'],
 'Main_Belt_a

Because the median doesn't seem a aqequate measure for the distance between the catogries (it can be accidentely a far distance between the two categories), wie decided to rank by the mean distance. The mean was in most cases also close to the median so it seemed to be the better measure. 

In [54]:
smallest = sorted(smalles.items(), key=lambda x: x[1][1])

In [55]:
smallest

[('Article_Feedback_Pilot', ['2.0', '1.5788086452293095']),
 ('Living_people', ['2.0', '2.027277406073083']),
 ('Year_of_birth_unknown', ['2.0', '2.027277406073083']),
 ('Harvard_University_alumni', ['3.0', '2.779947229551451']),
 ('Year_of_birth_missing', ['3', '2.8037620742247076']),
 ('Fellows_of_the_Royal_Society', ['3.0', '2.8631801373481247']),
 ('People_from_New_York_City', ['3.0', '2.946673706441394']),
 ('Year_of_birth_missing_(living_people)', ['3.0', '2.9715489989462593']),
 ('English-language_films', ['3.0', '2.9899577167019027']),
 ('American_film_actors', ['3.0', '3.058139534883721']),
 ('American_Jews', ['3.0', '3.095036958817318']),
 ('American_films', ['3', '3.1020084566596196']),
 ('Black-and-white_films', ['4.0', '3.1617336152219875']),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
  ['3.0', '3.179175475687104']),
 ('American_military_personnel_of_World_War_II', ['3.0', '3.218816067653277']),
 ('British_films', ['4.0', '3.2272245762711864']

In [31]:
import time
start = time.time()
smallest = run_bfs('Year_of_birth_unknown', graph, ['English_television_actors'])
smallest
print(time.time()-start)

720.9978468418121


To speed up our calculations we tried to use _"@autojit"_ wich is part of the _"numba"_ library. This will convert our the code placed after _"@autojit"_ to a code that can be executed below the python executer and this way might speed up the process a lot as it can be executed directly on the CPU. Some functions of course can't be converted, what will lead to checking with the python instance and the libraries loaded. This again might lead to a slower process, as happened in our case. With usigh _"@autojit"_ our _BFS_ ran 814.86 seconds, without 710.23 seconds using "Year_of_birth_unknown" as starting category and 'English_television_actors' as target category. Using _threating_ we could drop the runtime for these both categories to 642.2 secornds.

In [46]:
import queue as Q
from threading import Thread

# creating the list of input nodes here as this somehow doesn't seem to work with autojit 
# -> maybe use a (external) function to get it working
inp_nodes = [cat_nodes for cat_nodes in graph.nodes if graph.node[cat_nodes]['Year_of_birth_unknown']== True]




def bfs(graph, inp_cat, inp_nodes, dest_cat):
    #creating a dictionary for all the shortest paths for the nodes in inp_nodes
    sh_paths = {}
    # iterating over every node in the nodes of the input category in order to get the shortest distance for all those nodes
    for in_node in inp_nodes:
        # define a query we're working with
        queue = Q.Queue()
        # appending the node and its distance (as it's our starting node its 0) into the queue
        queue.put([in_node, 0])
        # defining a dictionary to check if we already visited the nodes of graph
        visited = {}
        # setting visited for all nodes in the graph to False
        for x in graph.nodes:
            visited[x] = False
        # set the distance to the destination category to infinitive. If we'll find a connection between
        # the input node and the destination category, we'll exchange inf with the distance
        sh_paths[in_node] = np.inf
        # work while our queue is not empty
        while queue.empty() != True:
            # get the first entry we put into the queue
            current = queue.get()
            # check if the entry we got from the queue is in the destinatino category and not in the input category
            if graph.node[current[0]][dest_cat] == True and graph.node[current[0]][inp_cat] != True:
                # if its true, set visited #uneccessary step as well break after anyway
                visited[current[0]]= True
                # add the shortest distance we found (current[1]) to a dictionary. 
                sh_paths[in_node] = current[1]
                # clear the queue as all the following nodes will have at least the same distance to our starting node
                queue.queue.clear()
            else:
                # get the successors of our current node (as its a directed graph)
                for i in graph.successors(current[0]):
                    # check if the successor is not visited
                    if visited[i] != True:
                        # if its not visited, put the found node in the queue, 
                        # together with the information about the distance it has from the starting node
                        queue.put([i, current[1]+1])
                # set the current point as visited so we don't add it to the queue again
                visited[current[0]]= True
    return sh_paths

'''medians = {}
for key in cats2.keys():  
    inp_cat = 'Year_of_birth_unknown'
    if inp_cat != key:
        sh_paths = bfs(graph, inp_cat, key) 
        medians[key] = median(sh_paths.values())
medians        
'''
start = time.time()
shortest = bfs(graph, 'Year_of_birth_unknown', inp_nodes, 'English_television_actors')
print(time.time() -start)
shortest
#smallest cat: Year_of_birth_unknown

710.2258560657501


{3335: 3,
 1656811: 3,
 10527: 3,
 170511: 4,
 16310: 3,
 22286: 3,
 23468: inf,
 23469: inf,
 23476: inf,
 24212: 4,
 1754435: 3,
 26206: 4,
 28993: 4,
 168219: 4,
 168261: 4,
 31093: 4,
 680787: inf,
 34422: 3,
 34424: 3,
 34425: 3,
 34762: 3,
 34909: 3,
 35263: 3,
 35264: inf,
 185814: 3,
 39892: 3,
 40716: inf,
 456791: inf,
 41699: 4,
 41778: 2,
 41761: 2,
 42011: 4,
 42539: 4,
 42400: inf,
 42090: 3,
 42245: 4,
 42669: inf,
 42246: 4,
 42303: 4,
 42370: 3,
 481329: 4,
 1703855: inf,
 42466: 3,
 42527: inf,
 43142: inf,
 42269: inf,
 41941: inf,
 42795: 4,
 42842: 3,
 53151: 2,
 42951: 3,
 46318: 3,
 43021: 4,
 43047: 3,
 43290: 3,
 43292: inf,
 43293: inf,
 43550: 3,
 43595: 4,
 43770: inf,
 43771: inf,
 44204: 3,
 44205: 3,
 44469: 3,
 342596: 3,
 1203095: 2,
 45522: inf,
 45524: inf,
 45897: inf,
 45523: inf,
 45948: inf,
 45950: inf,
 45952: inf,
 45975: 3,
 46821: 4,
 46826: 4,
 47007: 3,
 47017: 3,
 47085: 4,
 173549: 4,
 168239: 2,
 48145: 3,
 1202803: 3,
 48192: 3,
 48200:

In [296]:
def bfs(graph, in_node, all_nodes):
    queue = [[in_node, 0]]
    visited = defaultdict(lambda: (False, -1))
    for x in all_nodes:
        visited[x]
    while len(queue) != 0:
        current = queue.pop(0)
        for i in graph.successors(current[0]):
            if visited[i][0] != True:
                queue.append((i, current[1]+1))
        visited[current[0]]= (True, current[1])
    return visited

In [57]:
d = {}
d['a'] = 1
d['b'] = 3
d['c'] = np.inf
d['d'] = np.inf
d
median(d.values())

inf

In [165]:
cat = ['English_footballers']
def get_subgraph2(graph, cat_list):
    nodes = []
    for cat in cat_list:
        nodes.append(nodes for nodes in graph.nodes if graph.node[nodes][cat]== True)
    sub_g = graph.subgraph(nodes)
    return sub_g

In [32]:
def get_subgraph(graph, cat):
    sub_g = graph.subgraph([nodes for nodes in graph.nodes if graph.node[nodes][cat]== True])
    return sub_g

In [34]:
sub  = get_subgraph(graph, 'Year_of_birth_unknown')

In [35]:
nx.info(sub)

'Name: \nType: MultiDiGraph\nNumber of nodes: 2536\nNumber of edges: 1494\nAverage in degree:   0.5891\nAverage out degree:   0.5891'

In [14]:
graph.node[52]

AdjacencyView({401135: {0: {}}, 1069112: {0: {}}, 1163551: {0: {}}})

In [13]:
def no_in_edges(node):
    x = 0
    for i in graph.predecessors(node):
        x +=1
    return x

In [ ]:
graph.node[int(node)][key] = True

In [59]:
def article_score_cat(graph, nodes):
    nodes = [nodes for nodes in graph.nodes if graph.node[nodes][cat]== True]
    for node in nodes:    
        graph.node[node]['score'] = graph.node[node]['score'] +no_in_edges(node)
    

In [27]:
article_score(graph, 'Year_of_birth_unknown')

In [ ]:
def article_score(graph, catlist):
    for i in range(len(cat_list)):
        if i == 0:
            nodes = [nodes for nodes in graph.nodes if graph.node[nodes][cat[i]]== True]
            article_score_cat(graph, nodes)
        else:            
            nodes_c1 = [nodes for nodes in graph.nodes if graph.node[nodes][cat[i-1]]== True]
            nodes_c2 = [nodes for nodes in graph.nodes if graph.node[nodes][cat[1]]== True]
            article_score_cat(graph, nodes_c2)
            

            for node_1 in nodes_c1:
                for node_2 in nodes_c2:
                    if node_2 in graph.successors(node_1) and graph.node[node_2][before_cat] != True:
                        graph.node[node2]['score'] = graph.node[node2]['score'] + graph.node[node1]['score']

In [ ]:
def article_score(graph, catlist):
    for i in range(len(cat_list)):
        if i == 0:
            nodes = [nodes for nodes in graph.nodes if graph.node[nodes][cat[i]]== True]
            article_score_cat(graph, nodes)
        else:            
            nodes_c1 = [nodes for nodes in graph.nodes if graph.node[nodes][cat[i-1]]== True]
            nodes = [nodes for nodes in graph.nodes if graph.node[nodes][cat[i]]== True]
            article_score_cat(graph, nodes_c2)
            for node in nodes:
                for pred in graph.predecessors(node):
                    if graph.node[pred][cat[i-1]] == True:
                        graph.node[node]['score'] = graph.node[node]['score'] + graph.node[pred]['score']


categorien aufräumen => knoten für subgraph aus .values() beziehen => recherche ob attribute von knoten, die im subgraph gegeben werden, auch für den master graph gelten => bei jeder interation über kategorie neue knoten für subgraph hinzufügen und scoringfunktion ausführen

In [60]:
nodes = []
for i in cats2.keys():
    nodes.append([nodes for nodes in graph.nodes if graph.node[nodes][i]== True])
nodes

[[82897,
  81994,
  83423,
  79830,
  80741,
  22860,
  73150,
  78535,
  79858,
  80952,
  80953,
  81486,
  81836,
  81870,
  82719,
  85404,
  80389,
  81878,
  82385,
  81941,
  81945,
  79371,
  82346,
  82364,
  28961,
  80033,
  80513,
  83970,
  82753,
  82953,
  83868,
  82082,
  78937,
  82393,
  33973,
  82069,
  1449455,
  81996,
  82903,
  84458,
  41141,
  81847,
  668731,
  81214,
  82368,
  87360,
  48583,
  77463,
  48718,
  48730,
  84425,
  84427,
  751497,
  81619,
  84041,
  668803,
  81866,
  78097,
  78099,
  72482,
  72488,
  672513,
  77613,
  72493,
  72496,
  80740,
  81219,
  82587,
  72532,
  75773,
  76848,
  84668,
  84670,
  72536,
  79313,
  90028,
  72539,
  79183,
  81077,
  72545,
  72546,
  81828,
  78569,
  72563,
  72579,
  72580,
  72589,
  72591,
  77627,
  72592,
  72593,
  72607,
  77628,
  72595,
  72605,
  530297,
  72609,
  72613,
  80159,
  81129,
  81242,
  666150,
  81136,
  72616,
  78897,
  83042,
  83888,
  72617,
  72620,
  72629,
  

In [61]:
sub_t = graph.subgraph(nodes)

NetworkXError: Node [82897, 81994, 83423, 79830, 80741, 22860, 73150, 78535, 79858, 80952, 80953, 81486, 81836, 81870, 82719, 85404, 80389, 81878, 82385, 81941, 81945, 79371, 82346, 82364, 28961, 80033, 80513, 83970, 82753, 82953, 83868, 82082, 78937, 82393, 33973, 82069, 1449455, 81996, 82903, 84458, 41141, 81847, 668731, 81214, 82368, 87360, 48583, 77463, 48718, 48730, 84425, 84427, 751497, 81619, 84041, 668803, 81866, 78097, 78099, 72482, 72488, 672513, 77613, 72493, 72496, 80740, 81219, 82587, 72532, 75773, 76848, 84668, 84670, 72536, 79313, 90028, 72539, 79183, 81077, 72545, 72546, 81828, 78569, 72563, 72579, 72580, 72589, 72591, 77627, 72592, 72593, 72607, 77628, 72595, 72605, 530297, 72609, 72613, 80159, 81129, 81242, 666150, 81136, 72616, 78897, 83042, 83888, 72617, 72620, 72629, 81871, 72630, 83245, 94124, 72633, 653391, 80156, 82471, 72702, 72654, 72655, 72656, 72772, 72805, 72825, 83404, 89497, 89785, 72829, 72657, 72658, 84607, 84709, 669618, 72660, 72746, 72780, 72808, 726